In [1]:
#=========Data===========
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
#=========Model==========
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier

#=======Đánh giá========
from sklearn.model_selection import KFold
from sklearn.metrics import ConfusionMatrixDisplay, precision_recall_fscore_support, precision_score, recall_score, f1_score, accuracy_score
import time

In [2]:
#Đọc dữ liệu từ file CSV
df = pd.read_csv('data.csv')
#Bỏ Cột ID
if 'building_id' in df.columns:
    df = df.drop(columns=['building_id'])

#Chọn cột đặc trưng và nhãn
X = df.drop(columns=['labels'])
y = df['labels']

#One-hot encoding cho cột đặc trưng
X = pd.get_dummies(X)

#Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#=========Model==========
start_time = time.time()
modelRF = RandomForestClassifier(n_estimators=700, criterion='entropy', max_depth=30, min_samples_leaf=5, min_samples_split=10, random_state=42)
modelRF.fit(X_train, y_train)
end_time = time.time()
train_time = end_time - start_time

#Dự đoán trên tập kiểm tra
start_time = time.time()
y_pred = modelRF.predict(X_test)
end_time = time.time()
predict_time = end_time - start_time

#Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
f1_score_value = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

#Hiển thị kết quả
print("Kết quả đánh giá mô hình RandomForest:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"F1-score:  {f1_score_value:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"Thời gian huấn luyện: {train_time:.4f} giây")
print(f"Thời gian test:       {predict_time:.4f} giây")

Kết quả đánh giá mô hình RandomForest:
Accuracy:  0.7051
F1-score:  0.6879
Precision: 0.7169
Recall:    0.7051
Thời gian huấn luyện: 224.1138 giây
Thời gian test:       8.2675 giây


In [ ]:
# ======= KIỂM TRA OVERFITTING =======

# 1. So sánh hiệu suất trên tập train và test
print("=== KIỂM TRA OVERFITTING ===\n")

# Dự đoán trên tập huấn luyện
y_train_pred = modelRF.predict(X_train)

# Tính các chỉ số cho tập train
train_accuracy = accuracy_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred, average='weighted')
train_precision = precision_score(y_train, y_train_pred, average='weighted')
train_recall = recall_score(y_train, y_train_pred, average='weighted')

# So sánh kết quả
print("1. SO SÁNH HIỆU SUẤT TRAIN vs TEST:")
print(f"{'Metric':<12} {'Train':<10} {'Test':<10} {'Difference':<12}")
print("-" * 50)
print(f"{'Accuracy':<12} {train_accuracy:<10.4f} {accuracy:<10.4f} {train_accuracy-accuracy:<12.4f}")
print(f"{'F1-score':<12} {train_f1:<10.4f} {f1_score_value:<10.4f} {train_f1-f1_score_value:<12.4f}")
print(f"{'Precision':<12} {train_precision:<10.4f} {precision:<10.4f} {train_precision-precision:<12.4f}")
print(f"{'Recall':<12} {train_recall:<10.4f} {recall:<10.4f} {train_recall-recall:<12.4f}")

# Đánh giá overfitting
accuracy_diff = train_accuracy - accuracy
print(f"\n2. ĐÁNH GIÁ OVERFITTING:")
if accuracy_diff > 0.1:
    print(f"OVERFITTING NGHIÊM TRỌNG: Chênh lệch accuracy = {accuracy_diff:.4f} (>10%)")
elif accuracy_diff > 0.05:
    print(f"OVERFITTING VỪA PHẢI: Chênh lệch accuracy = {accuracy_diff:.4f} (5-10%)")
elif accuracy_diff > 0.02:
    print(f"OVERFITTING NHẸ: Chênh lệch accuracy = {accuracy_diff:.4f} (2-5%)")
else:
    print(f"KHÔNG OVERFITTING: Chênh lệch accuracy = {accuracy_diff:.4f} (<2%)")

=== KIỂM TRA OVERFITTING ===

1. SO SÁNH HIỆU SUẤT TRAIN vs TEST:
Metric       Train      Test       Difference  
--------------------------------------------------
Accuracy     0.7528     0.7051     0.0478      
F1-score     0.7401     0.6879     0.0522      
Precision    0.7684     0.7169     0.0515      
Recall       0.7528     0.7051     0.0478      

2. ĐÁNH GIÁ OVERFITTING:
⚠️  OVERFITTING NHẸ: Chênh lệch accuracy = 0.0478 (2-5%)
1. SO SÁNH HIỆU SUẤT TRAIN vs TEST:
Metric       Train      Test       Difference  
--------------------------------------------------
Accuracy     0.7528     0.7051     0.0478      
F1-score     0.7401     0.6879     0.0522      
Precision    0.7684     0.7169     0.0515      
Recall       0.7528     0.7051     0.0478      

2. ĐÁNH GIÁ OVERFITTING:
⚠️  OVERFITTING NHẸ: Chênh lệch accuracy = 0.0478 (2-5%)


In [4]:
# ======= TÓM TẮT VÀ KHUYẾN NGHỊ =======
print("\n" + "="*60)
print("           TÓM TẮT ĐÁNH GIÁ OVERFITTING")
print("="*60)

print("\n📊 CÁC CHỈ SỐ KIỂM TRA:")
print(f"1. Chênh lệch Train-Test Accuracy: {train_accuracy-accuracy:.4f}")

print("\n💡 KHUYẾN NGHỊ:")

# Đưa ra khuyến nghị dựa trên kết quả
if train_accuracy - accuracy > 0.1:
    print("🔴 MÔ HÌNH BỊ OVERFITTING NGHIÊM TRỌNG!")
    print("   → Giảm max_depth (từ 50 xuống 20-30)")
    print("   → Giảm n_estimators (từ 1500 xuống 500-1000)")
    print("   → Tăng min_samples_split và min_samples_leaf")
    print("   → Sử dụng feature selection")
    
elif train_accuracy - accuracy > 0.05:
    print("🟡 MÔ HÌNH CÓ DẤU HIỆU OVERFITTING VỪA PHẢI")
    print("   → Điều chỉnh nhẹ max_depth (giảm 10-20%)")
    print("   → Thêm regularization parameters")
    print("   → Kiểm tra lại preprocessing")
    
    
else:
    print("🟢 MÔ HÌNH HOẠT ĐỘNG TỐT!")
    print("   → Có thể thử tăng nhẹ complexity để cải thiện performance")
    print("   → Tập trung vào feature engineering")
    print("   → Thử ensemble với các mô hình khác")

print("\n🎯 THAM SỐ ĐỀ XUẤT CHO RANDOM FOREST:")
if train_accuracy - accuracy > 0.08:
    print("   n_estimators: 300-500")
    print("   max_depth: 15-25") 
    print("   min_samples_split: 10-20")
    print("   min_samples_leaf: 5-10")
else:
    print("   n_estimators: 500-1000")
    print("   max_depth: 25-40")
    print("   min_samples_split: 5-10") 
    print("   min_samples_leaf: 2-5")

print("\n" + "="*60)


           TÓM TẮT ĐÁNH GIÁ OVERFITTING

📊 CÁC CHỈ SỐ KIỂM TRA:
1. Chênh lệch Train-Test Accuracy: 0.0478

💡 KHUYẾN NGHỊ:
🟢 MÔ HÌNH HOẠT ĐỘNG TỐT!
   → Có thể thử tăng nhẹ complexity để cải thiện performance
   → Tập trung vào feature engineering
   → Thử ensemble với các mô hình khác

🎯 THAM SỐ ĐỀ XUẤT CHO RANDOM FOREST:
   n_estimators: 500-1000
   max_depth: 25-40
   min_samples_split: 5-10
   min_samples_leaf: 2-5

